# Development Notebook for "Race Track" Inference Demo

This demo makes use of jupyter widgets in order to display the simulated race between devices running inference on a dataset. You will need to use a jupyter notebook (or jupyter lab with extra configuration) in order for the jupyter widgets to display in your environment.

This code/demo is still under development by Andres Meza (anmeza@ucsd.edu).  

In [6]:
import ipywidgets as widgets
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

from PIL import Image
from PIL import ImageFont
from PIL import ImageDraw

import logging
import threading
import time

In [7]:
#I: Utility Functions
def get_html_heading(text, text_align="center"):
    html_style=f'\"font-family:monospace,monospace;color:black;font-size:40px;text-align:{text_align};font-weight: bold\"'
    return f"<h1 style={html_style}>{text}</h1>"

def get_html_label(text, text_align="center", font_size="30px"):
    html_style=f'\"font-family:monospace,monospace;color:black;font-size:{font_size};text-align:{text_align};font-weight: bold\"'
    return f"<h5 style={html_style}>{text}</h5>"

def generate_test_race_track():
    img_paths = list()
    for i in range(200):
        img = Image.new('RGB', (500,500), (250,250,250))
        draw = ImageDraw.Draw(img)
        font = ImageFont.truetype("ARIALN.TTF", 400)
        if i>=10:
            draw.text((80, 0),str(i),(0,0,0), font=font)
        else:
            draw.text((150, 0),str(i),(0,0,0), font=font)
        img.save('digit_number_img_'+str(i)+'.jpg')
        img_paths.append('digit_number_img_'+str(i)+'.jpg')
    return img_paths

#I: Racer class
class Racer:
    def __init__(self, name, inf_time, default_img_path):
        self.name        = name
        self.inf_time    = inf_time
        self.default_img = open(default_img_path, "rb").read()
        self.build_ui()

        
    def build_ui(self):
        #S: Build the racer's name label
        self.uie_name = widgets.HTML(value = get_html_heading(self.name, text_align="left"),
                                     layout=widgets.Layout(width='100%',
                                                           padding="0px 0px 0px 0px", 
                                                           margin="0px auto 0px auto"))
        
        #S: Build the racer's standing 🏆
        self.uie_standing = widgets.HTML(value = get_html_label("--/-- 🏆", text_align="right", font_size="20px"),
                                   layout=widgets.Layout(width='100%',
                                                         padding="10px 0px 0px 0px", 
                                                         margin="0px auto 0px auto"))
        
        
        #S: Build horizontal container for the racer's name and standing labels  
        self.uie_racer_info = widgets.HBox([self.uie_name, self.uie_standing],
                                          layout=widgets.Layout(width='100%', 
                                                                height='auto',
                                                                padding="0px 0px 0px 0px",
                                                                margin="0px 0px 0px 0px"))
        
        #S: Build the racer's image element and
        #S# set it to the provided default image 
        self.uie_img = widgets.Image(value=self.default_img,
                                     format='jpg',
                                     width=300,
                                     height=400,
                                     layout=widgets.Layout(padding="0px 0px 0px 0px", 
                                                           margin="50px auto 0px auto"))
        
        
        #S: Build the racer's progress label 
        self.uie_pbar_lbl_prcnt = widgets.HTML(value = get_html_label("0%", text_align="right"),
                                   layout=widgets.Layout(width='100%',
                                                         padding="0px 0px 0px 0px", 
                                                         margin="0px auto 0px auto"))
        
        
        #S: Build the racer's state label
        self.uie_pbar_lbl_state = widgets.HTML(value = get_html_label("🚦 Waiting", text_align="left"),
                                               layout=widgets.Layout(width='100%',
                                                                     padding="0px 0px 0px 0px",
                                                                     margin="0px auto 0px auto"))

        #S: Build horizontal container for the racer's state and progress labels  
        self.uie_pbar_lbls = widgets.HBox([self.uie_pbar_lbl_state, self.uie_pbar_lbl_prcnt],
                                          layout=widgets.Layout(width='100%', 
                                                                height='auto',
                                                                padding="0px 0px 0px 0px",
                                                                margin="0px 0px 0px 0px"))
        
        #S: Build the racer's progress bar
        self.uie_pbar = widgets.FloatProgress(
            value=0,
            min=0,
            max=1.0,
            description='',
            bar_style='info',
            style={'bar_color': '#207097'},
            orientation='horizontal',
            layout=widgets.Layout(width='100%',
                                  padding="0px 0px 0px 0px", 
                                  margin="50px auto 0px auto")
        )
        
        #S: Build the top level container for the racer's ui elements
        self.uie = widgets.VBox([self.uie_racer_info,
                                 self.uie_img, 
                                 self.uie_pbar, 
                                 self.uie_pbar_lbls
                                ],
                                layout=widgets.Layout(width='100%', 
                                                      padding="0px 10px 0px 10px", 
                                                      margin="0px 10px 0px 0px"),
                                box_style="info")
        
    def get_ui(self):
        return self.uie
    
    def update_uie_standing(self, standing, total_racers):
        if standing <= total_racers:
            #S: Generate well-formatted standing string following 
            #S# this template "<racer's place>/<# of racers> 🏆"
            stnd_str = f"{standing}".rjust(len(str(total_racers)), '0') + f"/{total_racers} 🏆"
            
            #S: Update the ui element displaying the racer's standing
            self.uie_standing.value = get_html_label(stnd_str, text_align="right", font_size="20px")
            
    def race_update(self, racer_info, track_info):
        #S: Unpack race information
        r_state, r_standing, total_racers = racer_info
        track_position, track_length, img_path = track_info
        
        #S: Display current image being evaluated
        self.uie_img.value = open(img_path, "rb").read()

        #S: Update ui elements displaying racer's progress 
        self.uie_pbar.value = (track_position+1)/track_length
        self.uie_pbar_lbl_prcnt.value = get_html_label(f"{round((track_position+1)/track_length*100, 1)}%", text_align="right")
        
        #S: Update the racer's standing
        self.update_uie_standing(r_standing, total_racers)
        
        if  1 > self.uie_pbar.value > 0:
            #S: Update the ui element displaying the racer's state
            #S# to running
            self.uie_pbar_lbl_state.value = get_html_label("🏎️ Running", text_align="left")
        elif self.uie_pbar.value == 1:
            #S: Update the ui element displaying the racer's state
            #S# to finished
            self.uie_pbar_lbl_state.value = get_html_label("🏁 Finished", text_align="left")             
        else:
            #S: Update the ui element displaying the racer's state
            #S# to unknown
            self.uie_pbar_lbl_state.value = get_html_label("⚠️ Unknown", text_align="left") 
        
    def race(self, race_track, log=False):
        #S: Configure logging object and log thread start 
        if log:    
            format = "%(asctime)s: %(message)s"
            logging.basicConfig(format=format, level=logging.INFO, datefmt="%H:%M:%S")
            logging.info("Racer \'%s\': starting", self.name)
        
        #S: Update the ui element displaying the racer's state
        #S# to running
        self.uie_pbar_lbl_state.value = get_html_label("🏎️ Running", text_align="left") 
        
        #S: Store the "track length" (i.e., the number of test images)
        #S# for use in determining racer's progress
        track_length = len(race_track)
        
        #S: "Run" inference on test images and update
        #S# ui elements accordingly
        for i, img_path in enumerate(race_track):
            #S: Display current image being evaluated
            self.uie_img.value = open(img_path, "rb").read()
            
            #S: Update ui elements displaying racer's progress 
            self.uie_pbar.value = (i+1)/track_length
            self.uie_pbar_lbl_prcnt.value = get_html_label(f"{round((i+1)/track_length*100, 1)}%", text_align="right")
            
            #S: Sleep to simulate inference time on actual device
            time.sleep(self.inf_time)
        
        #S: Update the ui element displaying the racer's state
        #S# to finished
        self.uie_pbar_lbl_state.value = get_html_label("🏁 Finished", text_align="left") 
        
        #S: Log thread finish
        if log:
            logging.info("Racer \'%s\': finishing", self.name)

class Race_Event:
    def __init__(self, racers, race_track):
        self.racers     = racers
        self.race_track = race_track
        self.build_ui()
        
    def build_ui(self):
        #S: Build the top level container for the race's ui elements
        self.uie = widgets.HBox([racer_x.get_ui() for racer_x in self.racers],
                                layout=widgets.Layout(width='99%', height='auto',
                                                      padding="0px 0px 0px 0px",
                                                      margin="0px 0px 0px 0px"))
        
        self.uie = widgets.GridBox([racer_x.get_ui() for racer_x in self.racers], 
                                   layout=widgets.Layout(grid_template_columns='48.5% 48.5%',
                                                         grid_template_rows='auto auto',
                                                         grid_gap='15px 1%'),
                                   box_style="")
        
    def get_ui(self):
        return self.uie
    
    def begin_race_v1(self, log=False):
        #S: Configure logging object and log race start 
        if log:
            format = "%(asctime)s: %(message)s"
            logging.basicConfig(format=format, level=logging.INFO, datefmt="%H:%M:%S")
            logging.info("Race: starting")
        
        #S: Initialize, store, and start racer threads
        threads = list()
        for racer_x in self.racers:
            x = threading.Thread(target=racer_x.race, args=(self.race_track,log))
            threads.append(x)
            x.start()
        
        #S: Run join on racer threads so this process waits
        #S# for all racers to finish before continuing
        for thread in threads:
            thread.join()
        
        #S: Log race finished
        if log:
            logging.info("Race: finishing")
           
        #S: Determine race standings
        r_inf_times = list()
        for i, r in enumerate(self.racers):
            r_inf_times.append((i, r.inf_time))
        r_inf_times.sort(key = lambda x: x[1], reverse=False)
        
        #S: Update racer's standings on ui
        total_racers = len(self.racers)
        for i, rit in enumerate(r_inf_times):
            r_index, _ = rit
            self.racers[r_index].update_uie_standing(i+1, total_racers)
            
    #TODO: Add in begin_race_v2(...) into this notebook

In [8]:
#S: Create a test "race track" (i.e. list of image file paths)
race_track = generate_test_race_track()


#S: Create Racers
racer1 = Racer("Racer 1", 0.01, "digit_number_img_0.jpg")
racer2 = Racer("Racer 2", 0.02, "digit_number_img_0.jpg")
racer3 = Racer("Racer 3", 0.03, "digit_number_img_0.jpg")
racer4 = Racer("Racer 4", 0.04, "digit_number_img_0.jpg")

#S: Create Race
race_event = Race_Event([racer1, racer2, racer3, racer4], race_track)

#S: Get race ui and display it (jupyter does this automatically)
race_event.get_ui()

GridBox(children=(VBox(box_style='info', children=(HBox(children=(HTML(value='<h1 style="font-family:monospace…

In [9]:
#S: Run race with logging
race_event.begin_race_v1(log=False)

In [5]:
#S: Run race with logging
race_event.begin_race_v1(log=True)

14:23:47: Race: starting
14:23:47: Racer 'Racer 1': starting
14:23:47: Racer 'Racer 2': starting
14:23:47: Racer 'Racer 3': starting
14:23:47: Racer 'Racer 4': starting
14:23:52: Racer 'Racer 1': finishing
14:23:53: Racer 'Racer 2': finishing
14:23:56: Racer 'Racer 3': finishing
14:23:57: Racer 'Racer 4': finishing
14:23:57: Race: finishing
